In [ ]:
from bs4 import BeautifulSoup                                       # HTML və XML-dən məlumat çıxarmaq üçün
from selenium import webdriver                                      # Brauzeri avtomatik idarə etmək üçün
from selenium.webdriver.chrome.service import Service               # ChromeDriver-i başlatmaq üçün
from selenium.webdriver.common.by import By                         # HTML elementlərini axtarmaq üçün (məs: By.CLASS_NAME)
from webdriver_manager.chrome import ChromeDriverManager            # ChromeDriver versiyasını avtomatik endirmək üçün
from selenium.webdriver.chrome.options import Options               # Chrome brauzerinə xüsusi ayarlar vermək üçün
from selenium.webdriver.support.ui import WebDriverWait             # Elementlərin yüklənməsini gözləmək üçün
from selenium.webdriver.support import expected_conditions as EC    # Müəyyən şərtlər ödənənə qədər gözləmək üçün
from selenium.common.exceptions import TimeoutException             # Vaxt aşımı xətasını idarə etmək üçün
from selenium.webdriver.common.keys import Keys                     # Klaviatura düymələrini simulyasiya etmək üçün (məs: ENTER)
import time                                                         # Kodda gecikmə vermək üçün (məs: time.sleep)
import pandas as pd                                                 # Verilənlərlə işləmək üçün (DataFrame, CSV və s.)
from tqdm import tqdm                                               # Dövr zamanı loading bar (proqress bar) göstərmək üçün
from IPython.display import display                                 # DataFrame-i səliqəli göstərmək üçün (xüsusilə Jupyter Notebook-da)
import re                                                           # Mətn təmizləmək, regex (müntəzəm ifadələr) ilə işləmək üçün
import pyodbc                                                       # SQL Server (və digər ODBC dəstəkləyən verilənlər bazaları) ilə bağlantı qurmaq üçün
from sqlalchemy import create_engine                                # SQLAlchemy ilə verilənlər bazasına bağlantı yaratmaq və pandas ilə işləmək üçün

### Brauzerin açılması və vebsayta daxil olunması

In [ ]:
# chrome driverden istifade
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
driver.maximize_window()

# esas sehifeye daxil olmaq
driver.get("https://bina.az/")

### Satış tipinin seçilməsi (kirayə, alış)

In [ ]:
is_type = int(input('Satış tipini seçmək istəyirsiz? Bəli --> 1 , Yox -- > 0 || Yoxsa Default olaraq Satış seçiləcək'))
if is_type == 1:
    time.sleep(1)

    # alis tipinin butonunun yuklenmesini gozlemek
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".search-row__cell--leased .selectric")))

    time.sleep(1)

    # alis tipinin butonuna klik
    sales_type_button = driver.find_element(By.CSS_SELECTOR, ".search-row__cell--leased .selectric").click()

    # alis tipinin siyahisi acilir
    sales_options = driver.find_elements(By.CSS_SELECTOR, "#new_q .search-row__cell--leased li")

    # siyahi indekslerle birlikde
    sales_list = []
    for i, option in enumerate(sales_options):
        sales_type_name = option.text.strip()
        if sales_type_name:
            sales_list.append((i, sales_type_name))


    df_sales_list = pd.DataFrame(sales_list, columns=["Indekslər", "Satis Tipi"])
    df_sales_list.set_index("Indekslər", inplace=True)
    display(df_sales_list)

    # userden satis tipinin secimini almaq
    choose_sales_options = int(input("Aşagıdakı listden uyğun dəyəri daxil edin!"))

    WebDriverWait(driver, 10).until(EC.element_to_be_clickable(sales_options[choose_sales_options]))

    # alis tipini seçirik
    sales_options[choose_sales_options].click()
else:
    pass

### Obyekt tipinin seçimi

In [ ]:
is_property = int(input('Əmlak tipini seçmək istəyirsiz? Bəli --> 1 , Yox -- > 0 || Yoxsa Default olaraq Mənzil seçiləcək'))
# obyekt tiplerinin siyahisinin yuklenmesini gozlemek 
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".search-row__cell--category .selectric-items ul li")))

# Obyektin tipi butonuna klik
property_type_button = driver.find_element(By.CSS_SELECTOR, ".search-row__cell--category .selectric").click()

# obyekt tiplerinin listi
property_options = driver.find_elements(By.CSS_SELECTOR, ".search-row__cell--category .selectric-items ul li")

# Əmlak tipi seçimi
choose_property = 0  # Default dəyər təyin et

if is_property == 1:
    # Əmlak tiplərini listə əlavə et
    property_list = []
    for i, property in enumerate(property_options):
        property_name = property.text.strip()
        if property_name:
                property_list.append((i, property_name))


    df_object_list = pd.DataFrame(property_list, columns=["Indekslər", "Emlak_Tipi"])
    df_object_list.set_index("Indekslər", inplace=True)
    display(df_object_list)

    choose_property = int(input("Aşagıdakı listden uyğun dəyəri daxil edin!"))
    property_options[choose_property].click()
else:
    property_options[0].click()

##### Obyektin tipinə görə seçimlər:
##### Mənzil, Yeni tikili, Köhnə tikili, Həyət evi/bağ evi, Ofis — Otaq seçimi.
##### Qaraj — Sahə seçimi.
##### Torpaq — Sahə, Sot.
##### Sahə — Sahə, m².

In [ ]:
# Secilen Obyekt tipine gore secimler 
if choose_property in [0, 1, 2, 3, 4]:
    is_number_room = int(input('Otaq sayı seçmək istəyirsiniz? Bəli --> 1 , Yox -- > 0 | Yoxsa default olaraq hamısı seçiləcək.'))
    if is_number_room == 1:
        # otaq sayi secimlerinin yuklenmesini gozlemek
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, ".search-row__cell--rooms .selectric-items > div")))

        # otaq sayi butonuna klik
        number_room_button = driver.find_element(By.CSS_SELECTOR, ".search-row__cell--rooms .selectric").click()

        # otaq sayi secimi daxil etmek
        choose_room = int(input("Istədiyiniz otaq sayına uyğun rəqəm daxil edin! | 1 Otaq üçün - 1 | 2 Otaq üçün - 2 | 3 Otaq üçün - 3 | 4 Otaq üçün - 4 | 5 və daha artıq üçün - 5"))
        if choose_room == 0:
            driver.find_element(By.CSS_SELECTOR, "#new_q li.selectric-reset-all.selected").click()
        if choose_room == 1:
            driver.find_element(By.CSS_SELECTOR, "#new_q .search-row__cell--rooms ul > li:nth-child(2)").click()
        elif choose_room == 2:
            driver.find_element(By.CSS_SELECTOR, "#new_q .search-row__cell--rooms li:nth-child(3)").click()
        elif choose_room == 3:
            driver.find_element(By.CSS_SELECTOR, "#new_q .search-row__cell--rooms li:nth-child(4)").click()
        elif choose_room == 4:
            driver.find_element(By.CSS_SELECTOR, "#new_q .search-row__cell--rooms li:nth-child(5)").click()
        elif choose_room == 5:
            driver.find_element(By.CSS_SELECTOR, "#new_q .search-row__cell--rooms li.last").click()
    else:
        pass
else:
    pass

if choose_property in [5, 6, 7]:
    is_min_value = int(input(" minimum sahə seçmək istəyirsiz? Bəli --> 1 , Yox -- > 0"))
    if is_min_value == 1:
        value_min = int(input('Minimum sahəni qeyd edin!'))
        # javascript ile deyer teyin olunur. "sahe" hissesi dinamik oldugu ucun 
        area_from = driver.find_element(By.CSS_SELECTOR, "#area_from")
        driver.execute_script("arguments[0].value = arguments[1]", area_from, str(value_min))
    else:
        pass
    
    is_max_value = int(input('Maksimum sahə təyin olunsun? Bəli --> 1 , Yox -- > 0'))
    if is_max_value == 1:
        value_max = int(input('Maksimum sahəni qeyd edin!'))
        # javascript ile deyer teyin olunur. "sahe" hissesi dinamik oldugu ucun 
        area_to = driver.find_element(By.CSS_SELECTOR, "#area_to")
        driver.execute_script("arguments[0].value = arguments[1]", area_to, str(value_max))
    else:
        pass
else:
    pass

### Qiymət seçimi

In [ ]:
# Minimum qiymət təyini
is_min_price = int(input('Minimum qiymət təyin etmək istəyirsiniz? Bəli --> 1 , Yox --> 0: '))
if is_min_price == 1:
    price_button = driver.find_element(By.CSS_SELECTOR, ".search-row__cell--price .search-row__input_label").click()
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#q_price_from")))
    choose_min_price = input("Minimum qiymət qeyd edin: ")
    min_price = driver.find_element(By.CSS_SELECTOR, "#q_price_from")
    min_price.clear()
    min_price.send_keys(choose_min_price)
else:
    pass

# Maksimum qiymət təyini
is_max_price = int(input('Maksimum qiymət təyin etmək istəyirsiniz? Bəli --> 1 , Yox --> 0: '))
if is_max_price == 1:
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#q_price_to")))
    choose_max_price = input("Maksimum qiymət qeyd edin: ")
    max_price = driver.find_element(By.CSS_SELECTOR, "#q_price_to")
    max_price.clear()
    max_price.send_keys(choose_max_price)
else:
    pass


### Şəhər seçimi 

In [ ]:
is_city = int(input('Şəhər seçimi etmək istəyirsiz? Bəli --> 1 , Yox -- > 0 || Yoxsa Default olaraq Bakı şəhəri seçiləcək'))
if is_city == 1:
    # seher secimlerinin acilmasi gozlenir
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR,("#new_q > div.search-row > div.search-row__cell.search-row__cell--city"))))

    # seher secimi (istenilen seher)
    city_button = driver.find_element(By.CSS_SELECTOR, "#new_q > div.search-row > div.search-row__cell.search-row__cell--city").click()

    # seherlerin listinin acilmasi gozlenilir
    WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located(((By.CSS_SELECTOR, ".selectric-items ul li"))))

    # Şəhərləri çıxart
    cities_options = driver.find_elements(By.CSS_SELECTOR, ".selectric-items ul li")


    # seherlerin indekslerle birge tapilmasi 
    city_list = []
    for i, city in enumerate(cities_options):
        city_name = city.text.strip()
        if city_name:
            city_list.append((i, city_name))


    # DataFrame yaratmaq
    df_city_list = pd.DataFrame(city_list, columns=["Indekslər", "Şəhərlər"])
    df_city_list.set_index("Indekslər", inplace=True)

    # # Bütün DataFrame-i çap etmək üçün
    display(df_city_list)

    # userden seher secimi almaq
    choose_city = int(input(f"Zəhmət olmasa aşağıdakı dataframe-dən Şəhərə uyğun İndeksi daxil edin!!! \
                            Məsələn: Bakı üçün: 22\t" ))


    cities_options[choose_city].click()
else:
    pass


# Ətraflı axtarış seçimləri 

### Ətraflı axtarış düyməsinə klikləyib siyahını açır.

In [ ]:
# etrafli axtaris butonunun yuklenmesini gozleyir
WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#js-search-row-filters-btn")))

# etrafli axtaris butonuna klik edir
detailed_search_button = driver.find_element(By.CSS_SELECTOR, "#js-search-row-filters-btn").click()

### Elan nömrəsi

In [ ]:
is_id_number = int(input('Sadəcə Elan Nömrəsi ilə Axtarış etmək istəyirsiz? Bəli --> 1 , Yox -- > 0'))
if is_id_number == 1:
    choose_ann_id = int(input('Elan nömrəsi daxil edin!'))
    ann_id = driver.find_element(By.CSS_SELECTOR, "#item_id")
    ann_id.clear()
    ann_id.send_keys(choose_ann_id)
else:
    pass

### Təmirlidir yoxsa təmirsiz?

In [ ]:
if choose_property in [0, 1, 2, 3, 4, 7]:
    is_repaired = int(input('Təmir növünü seçmək istəyirsiz? Bəli --> 1 , Yox --> 0 (Default: Hamısı seçiləcək): '))
    if is_repaired == 1:
        repair_type = int(input('Təmirli üçün --> 1 , Təmirsiz üçün --> 0: '))
        if repair_type == 1:
            driver.find_element(By.CSS_SELECTOR, "#js-search-filters > div.search-filters__body > div:nth-child(1) > div > label:nth-child(2) > span").click()
        elif repair_type == 0:
            driver.find_element(By.CSS_SELECTOR, "#js-search-filters > div.search-filters__body > div:nth-child(1) > div > label:nth-child(3) > span").click()
        else:
            print("Yalnız 0 və ya 1 daxil edə bilərsiniz.")
    else:
        # Default olaraq "Hamısı" seçilir (ilk seçim)
        driver.find_element(By.CSS_SELECTOR, "#js-search-filters > div.search-filters__body > div:nth-child(1) > div > label:nth-child(1) > span").click()

### Sahə seçimi

In [ ]:
if choose_property in [0, 1, 2, 3, 4]:
    field = int(input('Sahə təyin etmək istəyirsiz? Bəli --> 1 , Yox -- > 0'))
    if field == 1:
        # minimum sahe
        is_min_field = int(input('Minimum sahə təyin olunsun? Bəli --> 1 , Yox -- > 0'))
        if is_min_field == 1:
            choose_min_field = int(input('Minimum sahəni qeyd edin!'))
            min_field = driver.find_element(By.CSS_SELECTOR, "#filters-area-from")
            min_field.clear()
            min_field.send_keys(choose_min_field)
        else:
            pass

        # maksimum sahe
        is_max_field = int(input('Maksimum sahə təyin olunsun? Bəli --> 1 , Yox -- > 0'))
        if is_max_field == 1:
            choose_max_field = int(input('Maksimum sahəni qeyd edin!'))
            max_field = driver.find_element(By.CSS_SELECTOR, "#filters-area-to")
            max_field.clear()
            max_field.send_keys(choose_max_field)
        else:
            pass
    else:
        pass

### Torpaq sahəsi seçimi

In [ ]:
if choose_property == 3:
    land = int(input('Torpaq sahəsi seçimi etmək istəyirsiz? Bəli --> 1 , Yox -- > 0'))
    if land == 1:
        is_min_land = int(input('Minimum sahə seçmək istəyirsiz? Bəli --> 1 , Yox -- > 0'))
        if is_min_land == 1:
            choose_min_land = int(input('Minimum sahəni qeyd edin!'))
            min_land = driver.find_element(By.CSS_SELECTOR, "#q_land_area_from")
            min_land.clear()
            min_land.send_keys(choose_min_land)
        else:
            pass

        is_max_land = int(input('Maksimum sahə seçmək istəyirsiz? Bəli --> 1 , Yox -- > 0'))
        if is_max_land == 1:
            choose_max_land = int(input('Maksimum sahəni qeyd edin!'))
            max_land = driver.find_element(By.CSS_SELECTOR, "#q_land_area_to")
            max_land.clear()
            max_land.send_keys(choose_max_land)
        else:
            pass
    else:
        pass

### Mərtəbə seçimi

In [ ]:
# min mertebe
if choose_property in [0, 1, 2]:
    is_min_floor = int(input('Minimum mərtəbə təyin olunsun? Bəli --> 1 , Yox -- > 0'))
    if is_min_floor == 1:
        choose_min_floor = int(input('Minimum mərtəbə qeyd edin!'))
        min_floor = driver.find_element(By.CSS_SELECTOR, "#q_floor_from")
        min_floor.clear()
        min_floor.send_keys(choose_min_floor)
    else:
        pass

    # max mertebe
    is_max_floor = int(input('Maksimum mərtəbə təyin olunsun? Bəli --> 1 , Yox -- > 0'))
    if is_max_floor == 1:
        choose_max_floor = int(input('Maksimum mərtəbə qeyd edin!'))
        max_floor = driver.find_element(By.CSS_SELECTOR, "#q_floor_to")
        max_floor.clear()
        max_floor.send_keys(choose_max_floor)
    else:
        pass

### Tikinti şirkəti olsun?

In [ ]:
if choose_property in [0, 1, 4, 5, 7]:
    is_company = int(input('Tikinti şirkəti olsun? Bəli --> 1 , Yox -- > 0'))
    if is_company == 1:
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR,  ".search-filters__body > div:nth-child(2) .search-filters__checkboxes label > span")))
        company_button = driver.find_element(By.CSS_SELECTOR, ".search-filters__body > div:nth-child(2) .search-filters__checkboxes label > span").click()
    else:
        pass
else:
    pass

### Çıxarış

In [ ]:
is_title = int(input('Çıxarış olsun? Bəli --> 1 , Yox -- > 0'))
if is_title == 1:
    # Elementin yüklənməsini gözləmək
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".search-filters__body > div:nth-child(6) div > label:first-child > span")))
    title_element = driver.find_element(By.CSS_SELECTOR, ".search-filters__body > div:nth-child(6) div > label:first-child > span").click()
else:
    pass

### Ipoteka

In [ ]:
is_mortgage = int(input('Ipoteka olsun? Bəli --> 1 , Yox -- > 0'))
if is_mortgage == 1:
    WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".search-filters__body > div:nth-child(6) div > label:nth-child(2) > span")))
    mortgage_button = driver.find_element(By.CSS_SELECTOR, ".search-filters__body > div:nth-child(6) div > label:nth-child(2) > span").click()
else:
    pass

### Ofis üçün əlavə seçimlər

In [ ]:
if choose_property == 4:
    is_bussiness_centre = int(input('Biznes mərkəzi olsun? Bəli --> 1 , Yox -- > 0'))
    if is_bussiness_centre == 1:
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#js-search-filters .search-filters__checkboxes_row label:first-child span")))
        buss_center_button = driver.find_element(By.CSS_SELECTOR, "#js-search-filters .search-filters__checkboxes_row label:first-child span").click()
    else:
        pass

    is_house = int(input('Ev olsun? Bəli --> 1 , Yox -- > 0'))
    if is_house == 1:
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#js-search-filters .search-filters__checkboxes_row label:nth-child(2) span")))
        house_button = driver.find_element(By.CSS_SELECTOR, "#js-search-filters .search-filters__checkboxes_row label:nth-child(2) span").click()
    else:
        pass

    is_villa = int(input('Villa olsun? Bəli --> 1 , Yox -- > 0'))
    if is_villa == 1:
        WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, "#js-search-filters .search-filters__checkboxes_row label:nth-child(3) span")))
        villa_button = driver.find_element(By.CSS_SELECTOR, "#js-search-filters .search-filters__checkboxes_row label:nth-child(3) span").click()
    else:
        pass

### Axtarış etmək
 

In [ ]:
# axtaris duymesinin kodunun yuklenmesini gozlemek
WebDriverWait(driver, 10).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#new_q > div.search-row > div.search-row__cell.search-row__cell--btn")))

# axtaris duyməsinə klik et
search_button = driver.find_element(By.CSS_SELECTOR, "#new_q > div.search-row > div.search-row__cell.search-row__cell--btn")
search_button.click()
time.sleep(2)

### İstifadəçinin seçimlərinə əsasən formalaşan link

In [ ]:
# Seçimdən sonra yaranan URL-i al
time.sleep(3)  # Gözləmək lazımdır ki, URL dəyişsin
current_url = driver.current_url
print(current_url)

### Məlumatların çəkilməsi hissəsi

In [ ]:
# elanlar listi
list_data = []

# linklerin tekrarlanmasinin qarsisini almaq
see_link = set()

#----------------------------------------------------------------------------------------------------------------------------------------
# 1-den n-sayda elan axtaris ucun dovr
n = int(input("Neçə səhifə yoxlanilsin? "))
for i in tqdm(range(1, n + 1)):
    if "?" in current_url:
        url = f"{current_url}&page={i}"
    else:
        url = f"{current_url}?page={i}"


    #--------------------------------------------------------------------------
    # səhifənin yüklənməsi zamanı baş verə biləcək TimeoutException halları üçün 2 defə cəhd etmək
    for attempt in range(2):  
        try:
            driver.set_page_load_timeout(45)
            driver.get(url)
            break  # uğurlu alınsa, dövrü tərk et
        except TimeoutException:
            print(f"{i}. səhifə - Cəhd {attempt+1}: vaxt limiti keçdi")
            time.sleep(5)

    try:
        WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#js-items-search > div.items_list a")))
    except TimeoutException:
        print(f"{i}. səhifə yüklənmədi, keçilir...")
        continue

    time.sleep(1)


    #--------------------------------------------------------------------------
    # elan linklerinin tapilmasi
    elements = driver.find_elements(By.CSS_SELECTOR, "#js-items-search > div.items_list a")
    links = [e.get_attribute("href") for e in elements if e.get_attribute("href")]

    # tapilan elan linklerinden melumatlarin toplanmasi
    for link in links:
        if "/items/" not in link:  # yalnız item səhifələrini icazə ver
            continue

        # link gorulubse , kec
        if link in see_link:
            continue

        # Gorulmus linkler see_link - e elave edilir
        see_link.add(link)


        #--------------------------------------------------------------------------
        # səhifənin yüklənməsi zamanı baş verə biləcək TimeoutException halları üçün 2 defə cəhd etmək
        for attempt in range(2):  # maksimum 2 cəhd
            try:
                driver.set_page_load_timeout(45)
                driver.get(link)
                break
            except TimeoutException:
                print(f"Elan səhifəsi {link} - Cəhd {attempt+1}: vaxt limiti keçdi")
                time.sleep(5)
        
        
        #--------------------------------------------------------------------------
        # Elanın məlumatlarının yüklənməsini gözləyir
        try:
            WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.TAG_NAME, "h1")))
        except TimeoutException:
            print(f"Elan yüklənmədi: {link}")
            continue        
        

        #--------------------------------------------------------------------------
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        data = {"URL": link}


        #--------------------------------------------------------------------------
        # ID
        ann_id = soup.find("div", {"class":"product-actions__id"})
        data["ListingID"] = ann_id.text.strip() if ann_id else None

        #--------------------------------------------------------------------------
        # Elan adi
        announce = soup.find("h1", {"class":"product-title"})
        data["ListingName"] = announce.text.strip() if announce else None

        #--------------------------------------------------------------------------        
        # Əmlak tipini tapmaq üçün breadcrumb-dan istifadə edirik
        type_block = soup.find_all("a", {"class":"product-breadcrumbs__i-link"})
        if len(type_block) >= 2:
            data['SalesType'] = type_block[0].text.strip()  # Bu 'Satış'dır
        else:
            data['SalesType'] = None


        #--------------------------------------------------------------------------
        # Tarix 
        date = soup.find_all("span", {"class":"product-statistics__i-text"})
        if len(date) >= 2:
            data["ListingDate"] = date[1].text.strip()
        else:
            data["ListingDate"] = None
        
        #--------------------------------------------------------------------------
        # melumat blokunda olan datalar ucun (Kategoriya, Mertebe , otaq sayi vs.)
        # melumatlari saxlamaq ucun dictionary (key-value)
        data["CategoryID"] = None
        data["Floor"] = None
        data["FieldAreaSqm"] = None
        data["CountRoom"] = None
        data["Title"] = None
        data["Mortgage"] = None
        data["Repaired"] = None
        data["BuildingType (ForOffice)"] = None
        data["AreaInSot"] = None

        # melumat bloklarini tapmaq
        info_blok = soup.find_all("div", {"class":"product-properties__i"})

        # FƏRQLİ ADLARIN QARŞILIĞINI SAXLAYIRIQ
        label_mapping = {
            "Kateqoriya": "CategoryID",
            "Mərtəbə": "Floor",
            "Sahə": "FieldAreaSqm",
            "Otaq sayı": "CountRoom",
            "Çıxarış": "Title",
            "İpoteka": "Mortgage",
            "Təmir": "Repaired",
            "Binanın növü": "BuildingType (ForOffice)",
            "Torpaq sahəsi": "AreaInSot"}


        # her blokda basliq ve ona uygun deyerin tapilmasi
        for block in info_blok:
            # label_tag basliqlarin oldugu kod , span_tag deyerlerin oldugu kod
            label_tag = block.find("label", {"class":"product-properties__i-name"})
            span_tag = block.find("span", {"class":"product-properties__i-value"})
            
            # Əgər həm başlıq, həm də dəyər varsa:
            if label_tag and span_tag:
                label = label_tag.text.strip()  # Məs: "Kateqoriya"
                value = span_tag.text.strip()  # Məs: "Köhne tikili"
                
                # # Debug üçün - hansı etiketlərin olduğunu görmək üçün
                # print(f"Tapılan etiket: '{label}' - '{value}'")
                
                # Etiketləri uyğunlaşdırırıq
                if label in label_mapping:
                    data[label_mapping[label]] = value


        #--------------------------------------------------------------------------
        # Qiymət
        price = soup.find("span", {"class": "price-val"})
        data["Price"] = price.text.strip() if price else None


        #--------------------------------------------------------------------------
        # 1 kv.m üçün qiymet
        kv = soup.find_all("div", {"class":"product-price__i"})
        if len(kv) >= 2:
            data["PricePerSqm"] = kv[1].text.strip()
        else:
            data["PricePerSqm"] = None

        
        #--------------------------------------------------------------------------
        # Valyuta
        currecy = soup.find("span", {"class":"price-cur"})
        data["Currency"] = currecy.text.strip() if currecy else None


        #--------------------------------------------------------------------------
        # Unvan
        address = soup.find("div", {"class":"product-map__left__address"})
        data["Address"] = address.text.strip() if address else None

        #--------------------------------------------------------------------------
        # Yerləşdiyi ərazi
        location = soup.find("div", {"class":"location"})
        data['CityID'] = location.text.strip() if location else None

        #--------------------------------------------------------------------------
        # Satici (Şəxs)
        owner = soup.find("div", {"class":"product-owner__info-name"})
        data["SalesPerson"] = owner.text.strip() if owner else None


        #--------------------------------------------------------------------------
        # Satici novu
        owner_type = soup.find("div", {"class":"product-owner__info-region"})
        owner_type_2 = soup.find("div", {"class":"product-owner__residence-info-region"})

        if owner_type:
            data['SellerType'] = owner_type.text.strip()
        elif owner_type_2:
            data['SellerType'] = owner_type_2.text.strip()
        else:
            data['SellerType'] = None


        #--------------------------------------------------------------------------
        # Agentlik 
        agent = soup.find("div", {"class":"product-shop__owner-name"})
        data["Agency"] = agent.text.strip() if agent else None


        #--------------------------------------------------------------------------
        # Residence Adı
        residence = soup.find("div", {"class":"product-owner__residence-info-name"})
        data["Residence"] = residence.text.strip() if residence else None


        #--------------------------------------------------------------------------
        # elaqe nomresi
        contact = soup.find("div", {"class":"product-phones__btn-value"})
        data["Contact"] = contact.text.strip() if contact else None


        #--------------------------------------------------------------------------
        # Etiket
        label = soup.find("ul", {"class":"product-extras bz-d-flex bz-align-center bz-gap-15 bz-wrap-wrap"})
        data["Labels"] = label.text.strip() if label else None


        #--------------------------------------------------------------------------
        # Açıqlama
        description = soup.find("div", {"class":"product-description__content"})
        data["Description"] = description.text.strip() if description else None


        #--------------------------------------------------------------------------
        list_data.append(data)


        #--------------------------------------------------------------------------
        print(f"Sehife {i}. Ümumi {len(list_data)} elan toplandı.")

driver.quit()

#----------------------------------------------------------------------------------------------------------------------------------------
df = pd.DataFrame(list_data)
print("DataFrame yaradıldı!")

### Məlumatların təmizləməsi

In [ ]:
# ListingID - integer tipinə çevrilmə
df['ListingID'] = df['ListingID'].astype(str).str.replace('Elanın nömrəsi:', '', regex=True).astype(int)

# ListingName - string tipinə çevrilmə
df['ListingName'] = df['ListingName'].astype("string")

# Boolean tiplər
df[['Title', 'Mortgage', 'Repaired']] = df[['Title', 'Mortgage', 'Repaired']].astype(bool)

# SellerType - kategorik dəyərləri integer-ə çevirmə
df['SellerType'] = df['SellerType'].replace({"Yaşayış kompleksi": 3, "vasitəçi (agent)": 2, "mülkiyyətçi": 1})
df['SellerType'] = df['SellerType'].astype(int)

# Contact - string tipinə çevrilmə və əvəzləmə
df['Contact'] = df['Contact'].astype("string").str.replace("●●", "XX")

# Description - string tipinə çevrilmə və təmizləmə
df['Description'] = df['Description'].fillna('')
df["Description"] = df["Description"].apply(lambda x: re.sub(r'[^\w\s,.!?()-]', '', str(x)))
df['Description'] = df['Description'].apply(lambda x: re.sub(r'[\r\n]+', '. ', x).strip())
df['Description'] = df['Description'].astype("string")

# PricePerSqm - təmizləmə və integer-ə çevrilmə
df['PricePerSqm'] = df['PricePerSqm'].astype("string").str.replace('AZN', '').str.replace(r'/m[²2]', '', regex=True).str.replace(' ', '')
df = df[df['PricePerSqm'].str.isnumeric()]
df['PricePerSqm'] = df['PricePerSqm'].astype(int)

# CategoryID - kategorik dəyərləri integer-ə çevirmə
df['CategoryID'] = df['CategoryID'].replace({
    "Mənzil": 1, 
    "Yeni tikili": 2, 
    "Köhnə tikili": 3, 
    "Həyət evi/bağ evi": 4, 
    "Ofis": 5, 
    "Qaraj": 6, 
    "Torpaq": 7, 
    "Obyekt": 8
})
df['CategoryID'] = df['CategoryID'].astype(int)

# SalesType - kategorik dəyərləri integer-ə çevirmə
df['SalesType'] = df['SalesType'].replace({"Kirayə": 0, "Satış": 1})
df['SalesType'] = df['SalesType'].astype(int)

# FieldAreaSqm - təmizləmə və float-a çevrilmə
df['FieldAreaSqm'] = df['FieldAreaSqm'].astype(str).str.replace('m²', '', regex=False).str.strip()
df['FieldAreaSqm'] = df['FieldAreaSqm'].replace('', pd.NA).astype(float)

# CountRoom - integer tipinə çevrilmə
df['CountRoom'] = df['CountRoom'].astype(int)

# Floor - string tipinə çevrilmə
df['Floor'] = df['Floor'].astype("string")

# AreaInSot - float tipinə çevrilmə
df['AreaInSot'] = pd.to_numeric(df['AreaInSot'], errors='coerce')

# Price - təmizləmə və integer-ə çevrilmə
df['Price'] = df['Price'].astype(str).str.replace(' ', '').astype(int)

# String tipləri - NULL dəyərləri emal edilir
df['Residence'] = df['Residence'].fillna('').astype("string")
df['Labels'] = df['Labels'].fillna('').astype("string")
df['Agency'] = df['Agency'].fillna('').astype("string")
df['URL'] = df['URL'].astype("string")
df['BuildingType (ForOffice)'] = df['BuildingType (ForOffice)'].fillna('').astype("string")
df['Currency'] = df['Currency'].astype("string")
df['Address'] = df['Address'].astype("string")
# df['CityID'] = df['CityID'].astype("string")
df['SalesPerson'] = df['SalesPerson'].astype("string")


#------------------------------------------------------------------------------------------------------
# Şəhər adlarını ID-lərlə uyğunlaşdırmaq üçün xəritə (dictionary)
city_map = {
    "Ağcabədi": 17, "Ağdam": 18, "Ağdaş": 19, "Ağstafa": 20, "Ağsu": 21,
    "Astara": 22, "Bakı": 23, "Balakən": 24, "Beyləqan": 25, "Bərdə": 26,
    "Biləsuvar": 27, "Cəlilabad": 28, "Daşkəsən": 29, "Gədəbəy": 30,
    "Gəncə": 31, "Göyçay": 32, "Göygöl": 33, "Hacıqabul": 34, "Xaçmaz": 35,
    "Xankəndi": 36, "Xırdalan": 37, "Xızı": 38, "Xudat": 39, "İmişli": 40,
    "İsmayıllı": 41, "Kürdəmir": 42, "Qax": 43, "Qazax": 44, "Qəbələ": 45,
    "Qobustan": 46, "Quba": 47, "Qusar": 48, "Lerik": 49, "Lənkəran": 50,
    "Masallı": 51, "Mingəçevir": 52, "Naxçıvan": 53, "Naxçıvan MR": 54,
    "Neftçala": 55, "Oğuz": 56, "Saatlı": 57, "Sabirabad": 58, "Salyan": 59,
    "Samux": 60, "Siyəzən": 61, "Sumqayıt": 62, "Şabran": 63, "Şamaxı": 64,
    "Şəki": 65, "Şəmkir": 66, "Şirvan": 67, "Tərtər": 68, "Tovuz": 69,
    "Ucar": 70, "Yardımlı": 71, "Yevlax": 72, "Zaqatala": 73, "Zərdab": 74
}

# Əgər city_map-də tapılmazsa, default olaraq Bakı (23) ilə əvəz et. (Bakının müxtəlif rayonları, metrostasiyalar vs. üçün.)
df['CityID'] = df['CityID'].map(city_map).fillna(23).astype(int)

In [ ]:
df['ListingDate'] = df['ListingDate'].astype("string").str.replace('Yeniləndi:', '', regex=True)

df[['Date', 'Time']] = df['ListingDate'].str.split(', ', expand=True)

df['Date'] = pd.to_datetime(df['Date'].str.strip(), format='%d.%m.%Y')

df = df.drop('ListingDate', axis=1)

In [ ]:
pd.set_option('display.max_columns', None)
df

#### Məlumat tiplərinə baxış

In [ ]:
df.info()

# Dinamik dəyərlərin verilənlər bazasına köçürülməsi

### Microsoft SQL Server

In [ ]:
import pyodbc
from sqlalchemy import create_engine

conn = pyodbc.connect(
    "DRIVER={ODBC Driver 17 for SQL Server};"
    "SERVER=your-server-name;" 
    "DATABASE=RealEstateDB;"
    "Trusted_Connection=yes;" )
cursor = conn.cursor()

engine = create_engine(
    "mssql+pyodbc://localhost\\SQLEXPRESS/RealEstateDB?driver=ODBC+Driver+17+for+SQL+Server",
    connect_args={"trusted_connection": "yes"})

In [ ]:
# Duplicate-ları df-dən sil
df = df.drop_duplicates(subset=['ListingID'])

#--------------------------------------------------------------------------------------------------------------------------------------------------------------------

# NaN dəyərlərini emal etmək üçün bir funksiya yaradaq
def safe_value(value):
    if pd.isna(value):
        return None
    return value

#--------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Apartment kateqoriyaları: 1, 2, 3
if df["CategoryID"].isin([1, 2, 3]).any():
    insert_query_apartment = """
    INSERT INTO Dynamic.Apartment (ListingID, CategoryID, Floor, FieldAreaSqm, CountRoom, Title, Mortgage, Repaired)
    VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    """
    df_apartments = df[df["CategoryID"].isin([1, 2, 3])]
    for i, row in df_apartments.iterrows():
        try:
            cursor.execute(insert_query_apartment, (
                safe_value(row["ListingID"]), 
                safe_value(row["CategoryID"]), 
                safe_value(row["Floor"]), 
                safe_value(row["FieldAreaSqm"]), 
                safe_value(row["CountRoom"]), 
                safe_value(row["Title"]), 
                safe_value(row["Mortgage"]), 
                safe_value(row["Repaired"])
            ))
        except pyodbc.IntegrityError:
            pass

#--------------------------------------------------------------------------------------------------------------------------------------------------------------------

# House kateqoriyası: 4
if (df["CategoryID"] == 4).any():
    insert_query_house = """
    INSERT INTO Dynamic.House (ListingID, CategoryID, FieldAreaSqm, AreaInSot, CountRoom, Mortgage, Repaired)
    VALUES (?, ?, ?, ?, ?, ?, ?)
    """
    df_house = df[df["CategoryID"] == 4]
    for i, row in df_house.iterrows():
        try:
            cursor.execute(insert_query_house, (
                safe_value(row["ListingID"]), 
                safe_value(row["CategoryID"]), 
                safe_value(row["FieldAreaSqm"]), 
                safe_value(row["AreaInSot"]), 
                safe_value(row["CountRoom"]), 
                safe_value(row["Mortgage"]), 
                safe_value(row["Repaired"])
            ))
        except pyodbc.IntegrityError:
            pass

#--------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Office kateqoriyası: 5
if (df["CategoryID"] == 5).any():
    insert_query_office = """
    INSERT INTO Dynamic.Office (ListingID, CategoryID, FieldAreaSqm, [BuildingType (ForOffice)], CountRoom, Title, Repaired)
    VALUES (?, ?, ?, ?, ?, ?, ?)
    """
    df_office = df[df["CategoryID"] == 5]
    for i, row in df_office.iterrows():
        try:
            cursor.execute(insert_query_office, (
                safe_value(row["ListingID"]), 
                safe_value(row["CategoryID"]), 
                safe_value(row["FieldAreaSqm"]), 
                safe_value(row["BuildingType (ForOffice)"]), 
                safe_value(row["CountRoom"]), 
                safe_value(row["Title"]), 
                safe_value(row["Repaired"])
            ))
        except pyodbc.IntegrityError:
            pass

#--------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Garage kateqoriyası: 6
if (df["CategoryID"] == 6).any():
    insert_query_garage = """
    INSERT INTO Dynamic.Garage (ListingID, CategoryID, FieldAreaSqm, Title)
    VALUES (?, ?, ?, ?)
    """
    df_garage = df[df["CategoryID"] == 6]
    for i, row in df_garage.iterrows():
        try:
            cursor.execute(insert_query_garage, (
                safe_value(row["ListingID"]), 
                safe_value(row["CategoryID"]), 
                safe_value(row["FieldAreaSqm"]), 
                safe_value(row["Title"])
            ))
        except pyodbc.IntegrityError:
            pass

#--------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Land kateqoriyası: 7
if (df["CategoryID"] == 7).any():
    insert_query_land = """
    INSERT INTO Dynamic.Land (ListingID, CategoryID, FieldAreaSqm, Title, Mortgage)
    VALUES (?, ?, ?, ?, ?)
    """
    df_land = df[df["CategoryID"] == 7]
    for i, row in df_land.iterrows():
        try:
            cursor.execute(insert_query_land, (
                safe_value(row["ListingID"]), 
                safe_value(row["CategoryID"]), 
                safe_value(row["FieldAreaSqm"]), 
                safe_value(row["Title"]), 
                safe_value(row["Mortgage"])
            ))
        except pyodbc.IntegrityError:
            pass

#--------------------------------------------------------------------------------------------------------------------------------------------------------------------

# CommercialProperty kateqoriyası: 8
if (df["CategoryID"] == 8).any():
    insert_query_commercial = """
    INSERT INTO Dynamic.CommercialProperty (ListingID, CategoryID, FieldAreaSqm, Title, Mortgage, Repaired)
    VALUES (?, ?, ?, ?, ?, ?)
    """
    df_commercial = df[df["CategoryID"] == 8]
    for i, row in df_commercial.iterrows():
        try:
            cursor.execute(insert_query_commercial, (
                safe_value(row["ListingID"]), 
                safe_value(row["CategoryID"]), 
                safe_value(row["FieldAreaSqm"]), 
                safe_value(row["Title"]), 
                safe_value(row["Mortgage"]), 
                safe_value(row["Repaired"])
            ))
        except pyodbc.IntegrityError:
            pass

#--------------------------------------------------------------------------------------------------------------------------------------------------------------------

# EstateListing
insert_query_estatelisting = """
INSERT INTO Dynamic.EstateListing (ListingID, ListingName, SalesType, CategoryID, SellerType, Price, PricePerSqm, Currency, CityID, Address, Date, Time)
VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
"""
for i, row in df.iterrows():
    try:
        cursor.execute(insert_query_estatelisting, (
            safe_value(row["ListingID"]), 
            safe_value(row["ListingName"]), 
            safe_value(row["SalesType"]), 
            safe_value(row["CategoryID"]), 
            safe_value(row["Price"]), 
            safe_value(row["PricePerSqm"]),
            safe_value(row["Currency"]), 
            safe_value(row["City"]), 
            safe_value(row["Address"]), 
            safe_value(row["Date"]), 
            safe_value(row["Time"])
        ))
    except pyodbc.IntegrityError:
        pass

#--------------------------------------------------------------------------------------------------------------------------------------------------------------------

# SellerInfo - Xəta verən kod
insert_query_sellerinfo = """
INSERT INTO Dynamic.SellerInfo (ListingID, ListingName, Contact, SellerType, Agency, Residence)
VALUES (?, ?, ?, ?, ?, ?)
"""
for i, row in df.iterrows():
    try:
        cursor.execute(insert_query_sellerinfo, (
            safe_value(row["ListingID"]), 
            safe_value(row["ListingName"]), 
            safe_value(row["Contact"]), 
            safe_value(row["SellerType"]), 
            safe_value(row["Agency"]), 
            safe_value(row["Residence"])
        ))
    except pyodbc.IntegrityError:
        pass

#--------------------------------------------------------------------------------------------------------------------------------------------------------------------

# URL
insert_query_url = """
INSERT INTO Dynamic.URL (ListingID, URL)
VALUES (?, ?)
"""
for i, row in df.iterrows():
    try:
        cursor.execute(insert_query_url, (
            safe_value(row["ListingID"]), 
            safe_value(row["URL"])
        ))
    except pyodbc.IntegrityError:
        pass

#--------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Labels
insert_query_labels = """
INSERT INTO Dynamic.Labels (ListingID, Labels)
VALUES (?, ?)
"""
for i, row in df.iterrows():
    try:
        cursor.execute(insert_query_labels, (
            safe_value(row["ListingID"]), 
            safe_value(row["Labels"])
        ))
    except pyodbc.IntegrityError:
        pass

#--------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Description
insert_query_description = """
INSERT INTO Dynamic.Description (ListingID, Description)
VALUES (?, ?)
"""
for i, row in df.iterrows():
    try:
        cursor.execute(insert_query_description, (
            safe_value(row["ListingID"]), 
            safe_value(row["Description"])
        ))
    except pyodbc.IntegrityError:
        pass

#--------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Commit əməliyyatını yadda saxlayın
conn.commit()

# Bağlantını bağlayın
cursor.close()
conn.close()